In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve 
from sklearn.metrics import confusion_matrix

In [23]:
np.random.seed(20201101)

In [2]:
diabetes = pd.read_csv("diabetes.csv")

In [3]:
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Cambiamos las columnas de notación, a minúsculas y agregamos guión bajo. Está un poco elaborado porque tiene que checar las que son mayúsuculas porque ahí empieza una palabra nueva.

In [4]:
def rename_column(col):
    n = len(col)
    index = []
    for i in range(n-1):
        if col[i+1].isupper():
            index.append(i+1)
            
    col = col.lower()
    bias = 0
    for i in index:
        col = col[:(i+bias)] + '_' + col[(i+bias):]
        bias+=1
    return col

In [5]:
diabetes.rename(columns = {col: rename_column(col) for col in diabetes.columns.values}, inplace = True)

In [6]:
diabetes.head()

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,b_m_i,diabetes_pedigree_function,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
diabetes.dtypes

pregnancies                     int64
glucose                         int64
blood_pressure                  int64
skin_thickness                  int64
insulin                         int64
b_m_i                         float64
diabetes_pedigree_function    float64
age                             int64
outcome                         int64
dtype: object

Todas las varaible son numéricas entonces no tenemos problema con Scikit learn. Ahora, ponemos en la y la variable que queremos predecir.

In [9]:
y = diabetes['outcome']

# 1¿Cuál es la etiqueta positiva?

La etiqueta positiva es 1, es decir que sí tiene diabetes.

In [14]:
X = diabetes.filter(['pregnancies', 'glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'b_m_i', 'diabetes_pedigree_function', 'age'], axis = 1)

In [20]:
X.shape

(768, 8)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25)

No usamos random_state porque ya definimos la semilla al principio.

In [27]:
print("\nX_train, y_train: ", (X_train.shape, y_train.shape))
print("\nX_test, y_test: ", (X_test.shape, y_test.shape))


X_train, y_train:  ((576, 8), (576,))

X_test, y_test:  ((192, 8), (192,))


# 2 Divide el conjunto de datos para que el 25% de los datos sean de pruebas. ¿Cuántas observaciones tienes en entrenamiento y pruebas?

Tenemos 576 observaciones en entrenamiento, y 192 en pruebas.

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ParameterGrid, GridSearchCV

In [34]:
dt = DecisionTreeClassifier()
grid = {'min_samples_leaf': [3,5,7], 'max_depth': [5, 10, 15]}
gs = GridSearchCV(dt, param_grid = grid, scoring = 'precision', cv = 10, return_train_score = True)
gs.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [3, 5, 7]},
             return_train_score=True, scoring='precision')

# 3 ¿Cuántos modelos generarás?

En el grid se hace un producto cruz con los conjuntos de parámetros que escogemos, en este caso son 3 de min_samples_leaf, y 3 de max_depth, por lo tanto se generan 3*3 = 9 modelos

In [39]:
gs.best_estimator_

DecisionTreeClassifier(max_depth=5, min_samples_leaf=5)

Ahí vemos que el mejor modelo de acuerdo a la precision es cuando max_depth es 5 y cuando min_samples_leaf es 5. Entonces tomamos ése.

In [47]:
#Aquí no estoy seguro si se hace así, o tengo que crear otro det que se el best estimator.
dt.fit(X_train, y_train)

DecisionTreeClassifier()

>
>Reee: aquí no sé si está bien usar dt.fit así, o debería de usar el best estimator para contestar las preguntas 4 y 5*
>

In [45]:
feature_importances_df = pd.DataFrame({'feature': X.columns.values, 
                                       'importance': dt.feature_importances_})


In [46]:
feature_importances_df.sort_values(by="importance", ascending=False)

,feature,importance
1,glucose,0.335138
5,b_m_i,0.154671
6,diabetes_pedigree_function,0.134337
7,age,0.130641
2,blood_pressure,0.073693
4,insulin,0.071629
0,pregnancies,0.058590
3,skin_thickness,0.041301


# 4 ¿Cuál es la variable que está en la raíz?

Glucose, ya que es la que mayor importancia tiene.

# 5 ¿Cuáles son las 3 variables que aportan más información a la predicción?

Glucose, b_m_i, diabetes_pedigree_function

# Elimina las variables que tienen menos del 7% de importancia 

Entonces, hago una nueva X pero sin pregnancies y sin skin_thickness

In [48]:
X = diabetes.filter(['glucose', 'blood_pressure', 'insulin', 'b_m_i', 'diabetes_pedigree_function', 'age'], axis = 1)

# Vuelve a generar un GridSearchCV con los mismos hiperparámetros que configuraste anteriormente.

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25)
dt = DecisionTreeClassifier()
grid = {'min_samples_leaf': [3,5,7], 'max_depth': [5, 10, 15]}
gs = GridSearchCV(dt, param_grid = grid, scoring = 'precision', cv = 10, return_train_score = True)
gs.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [3, 5, 7]},
             return_train_score=True, scoring='precision')

In [51]:
gs.best_estimator_

DecisionTreeClassifier(max_depth=5, min_samples_leaf=7)

# 6 ¿Qué hiperparámetros tiene el mejor modelo?

Ahora cambió, best estimator es con max_depth = 5, y min_samples_leaf = 7

In [53]:
be = gs.best_estimator_

>
>
>Reee: aquí tampoco sé si está bien pero use el bes estimator para contestar las preguntas(7,8,9,10), le puse be

# 7¿Cuál es el punto de corte del modelo que cumple con las restricciones de negocio (4 decimales)?

Quermeos tener menos del 10 por ciento de False Positive

In [56]:
predictions_scores = be.predict_proba(X_test)
predictions_scores[:10]

array([[0.09615385, 0.90384615],
       [0.09615385, 0.90384615],
       [0.63636364, 0.36363636],
       [0.275     , 0.725     ],
       [0.97435897, 0.02564103],
       [0.59459459, 0.40540541],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.45454545, 0.54545455],
       [0.80769231, 0.19230769]])

In [58]:
fpr, tpr, thresholds_roc = roc_curve(y_test, predictions_scores[:,1], pos_label=1)

In [59]:
metricas_df = pd.DataFrame({'thresholds': thresholds_roc, 'tpr': tpr, 'fpr': fpr})
metricas_df['tnr'] = 1 - metricas_df['fpr']
metricas_df['fnr'] = 1 - metricas_df['tpr']
metricas_df['precision'] = metricas_df.tpr/(metricas_df.tpr + metricas_df.fpr)
metricas_df['recall'] = metricas_df.tpr/(metricas_df.tpr + metricas_df.fnr)
metricas_df

,thresholds,tpr,fpr,tnr,fnr,precision,recall
0,2.000000,0.000000,0.000000,1.000000,1.000000,NaN,0.000000
1,1.000000,0.057143,0.016393,0.983607,0.942857,0.777070,0.057143
2,0.903846,0.257143,0.040984,0.959016,0.742857,0.862529,0.257143
3,0.725000,0.400000,0.081967,0.918033,0.600000,0.829932,0.400000
4,0.666667,0.428571,0.098361,0.901639,0.571429,0.813333,0.428571
5,0.550000,0.428571,0.131148,0.868852,0.571429,0.765690,0.428571
6,0.545455,0.457143,0.147541,0.852459,0.542857,0.756003,0.457143
7,0.500000,0.485714,0.147541,0.852459,0.514286,0.767012,0.485714
8,0.444444,0.500000,0.163934,0.836066,0.500000,0.753086,0.500000
9,0.428571,0.528571,0.180328,0.819672,0.471429,0.745623,0.528571


Entonces el punto de corte es 0.6666 ya que con eso tenemos solo el 9.8 por ciento de false positive.

# 8 ¿Qué porcentaje de FNR tendremos?

Tendremos el 57.14 por ciento de False Negatives

# 9 ¿Qué eficiencia tiene el modelo en ese punto?

Recordamos que la eficiencia es igual a precision, entonces en ese punto tenemos 0.8133 de eficiencia.

# 10 ¿Cuál es la matriz de confusión asociada a ese punto de corte?

In [63]:

prediction_labels = [1 if score >= 0.6666 else 0 for score in predictions_scores[:,1]]

In [66]:
confusion_matrix(y_test, prediction_labels)

array([[110,  12],
       [ 40,  30]], dtype=int64)

Ahí está la matriz de confusión asociada a ese corte